In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
print(datetime.datetime.now())
print(os.getcwd())

2020-01-31 14:19:08.543640
/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1


In [2]:
df_mapping=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/aggregatd_to_FB_directly_PlanB/FB segments.xlsx")
print(df_mapping.sheet_names)
df_mapping=df_mapping.parse("df_0_to_18_count_all",dtype=str)
print(df_mapping.shape)
df_mapping.head(2)

['Segment Summary', 'df_0_to_18_count_all']
(272, 9)


,segment,Placement Name,Store Type,Shopper Status,HHI,Dwelling,customer_id_hashed,segment_T_name,test_ids
0,Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOrCo...,Legacy_NA_Rewards_Lapsed1318_Above50k_Multi_NA,Legacy,Lapsed1318,Above50k,MultiFamilyOrCondo,17460,T_Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOr...,16969
1,Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOrCo...,Legacy_NA_Rewards_Lapsed1318_Above50k_Multi_NA,Legacy,Lapsed1318,Above50k,MultiFamilyOrCondo,22417,T_Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOr...,21786


In [3]:
df_mapping=df_mapping[['segment_T_name','segment','Placement Name']].drop_duplicates()
df_mapping.shape

(272, 3)

In [4]:
for col in df_mapping.columns.tolist():
    print(col,df_mapping[col].nunique())

segment_T_name 272
segment 272
Placement Name 19


In [5]:
list_all_files=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/*.csv")

df_list_segs_detail=pd.DataFrame({"detail_path":list_all_files})
df_list_segs_detail['basename']=df_list_segs_detail['detail_path'].apply(lambda x: os.path.basename(x).split(".")[0])

df_mapping=pd.merge(df_mapping,df_list_segs_detail,left_on="segment_T_name",right_on="basename",how="left")
df_mapping=df_mapping[df_mapping['Placement Name']!="DON’T USE"]
df_mapping.shape

(270, 5)

In [6]:
dict_pattern_new_name=df_mapping.set_index("detail_path").to_dict()["Placement Name"]

a=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/aggregatd_to_FB_directly_PlanB/FB segments.xlsx")
a=a.parse('Segment Summary',dtype=str,skiprows=2)
list_name=a.iloc[:,8].unique().tolist()
list_name=[x for x in list_name if str(x)!="nan"]
list_name.sort()
print(list_name==sorted(df_mapping['Placement Name'].unique().tolist()))

# The name in the mapping file good to use

True


In [7]:
dict_list_of_df_SegOnFB={}

for k_path,v_new_name in dict_pattern_new_name.items():
    df=pd.read_csv(k_path,dtype=str)
    df['audience_on_FB']=v_new_name
    print(str(datetime.datetime.now())[:16],os.path.basename(k_path).split(".")[0],df.shape)
    if v_new_name in dict_list_of_df_SegOnFB.keys():
        dict_list_of_df_SegOnFB.update({v_new_name:dict_list_of_df_SegOnFB[v_new_name].append(df)})
    else:
        dict_list_of_df_SegOnFB.update({v_new_name:df})

2020-01-31 14:19 T_Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (16969, 4)
2020-01-31 14:19 T_Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1 (21786, 4)
2020-01-31 14:19 T_Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (10143, 4)
2020-01-31 14:19 T_Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1 (9636, 4)
2020-01-31 14:19 T_Legacy_Lapsed1318_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (318334, 4)
2020-01-31 14:19 T_Legacy_Lapsed1318_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (180432, 4)
2020-01-31 14:19 T_Legacy_Lapsed1318_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (247417, 4)
2020-01-31 14:19 T_Legacy_Lapsed1318_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (123622, 4)
2020-01-31 14:19 T_Legacy_Lapsed1318_L_S_Above50K_MultiFamilyOrCondo_Competit

2020-01-31 14:19 T_Legacy_Active_M_S_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (147723, 4)
2020-01-31 14:19 T_Legacy_Active_M_S_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (78194, 4)
2020-01-31 14:19 T_Legacy_Active_M_S_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (189114, 4)
2020-01-31 14:19 T_Legacy_Active_M_S_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (90492, 4)
2020-01-31 14:19 T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (27968, 4)
2020-01-31 14:19 T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1 (9596, 4)
2020-01-31 14:19 T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (11065, 4)
2020-01-31 14:19 T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1 (10590, 4)
2020-01-31 14:20 T_SOTF_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (486032, 4)
2020-01-31 

2020-01-31 14:21 T_Legacy_Active_H_P_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (203651, 4)
2020-01-31 14:21 T_Legacy_Active_H_P_Below50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (55728, 4)
2020-01-31 14:21 T_Legacy_Active_H_S_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (11271, 4)
2020-01-31 14:21 T_Legacy_Active_H_S_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1 (9051, 4)
2020-01-31 14:21 T_Legacy_Active_H_S_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (13016, 4)
2020-01-31 14:21 T_Legacy_Active_H_S_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1 (9469, 4)
2020-01-31 14:21 T_Legacy_Active_H_S_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (63264, 4)
2020-01-31 14:21 T_Legacy_Active_H_S_Below50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (22457, 4)
2020-01-31 14:21 T_Legacy_Active_H_S_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (91365, 4)
2020-

2020-01-31 14:22 T_Legacy_Active_H_P_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1 (264168, 4)
2020-01-31 14:22 T_Legacy_Active_H_P_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1 (153218, 4)
2020-01-31 14:22 T_Legacy_Active_H_P_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1 (227759, 4)
2020-01-31 14:22 T_Legacy_Active_H_P_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1 (90732, 4)
2020-01-31 14:22 T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1 (94494, 4)
2020-01-31 14:22 T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1 (45912, 4)
2020-01-31 14:22 T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1 (116026, 4)
2020-01-31 14:22 T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1 (42331, 4)
2020-01-31 14:22 T_Legacy_Active_L_P_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1 (77109, 4)
2020-01-31 14:22 T_Legacy_Active_L_P_Unknown_Unknown_CompetitorsHigh_Furinitu

In [24]:
df_count=pd.DataFrame()
for FB_Name,df_aggred_seg in dict_list_of_df_SegOnFB.items():
    df_aggred_seg.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/aggregatd_to_FB_directly_PlanB/"+FB_Name+".csv",index=False)
    df_count_seg=pd.DataFrame({"name":[FB_Name],"count":[df_aggred_seg.shape[0]]})
    df_count=df_count.append(df_count_seg)
    print(FB_Name,df_aggred_seg.shape)

Legacy_NA_Rewards_Lapsed1318_Above50k_Multi_NA (83828, 4)
Legacy_NA_Rewards_Lapsed1318_Above50k_Single_NA (1264030, 4)
SOTF_NA_Rewards_Lapsed1318_Above50k_Multi_NA (30874, 4)
SOTF_NA_Rewards_Lapsed1318_Above50k_Single_NA (533438, 4)
Legacy_NA_Rewards_Active_Above50k_Multi_NA (347795, 4)
Legacy_NA_Rewards_Active_Above50k_Single_NA (5436025, 4)
SOTF_NA_Rewards_Active_Above50k_Multi_NA (140127, 4)
SOTF_NA_Rewards_Active_Above50k_Single_NA (2520258, 4)
Legacy_NA_Rewards_Lapsed1318_Below50k_Multi_NA (98953, 4)
Legacy_NA_Rewards_Lapsed1318_Below50k_Single_NA (358593, 4)
SOTF_NA_Rewards_Lapsed1318_Below50k_Multi_NA (33092, 4)
SOTF_NA_Rewards_Lapsed1318_Below50k_Single_NA (139098, 4)
Legacy_NA_Rewards_Active_Below50k_Multi_NA (404633, 4)
Legacy_NA_Rewards_Active_Below50k_Single_NA (1547988, 4)
SOTF_NA_Rewards_Active_Below50k_Multi_NA (159504, 4)
SOTF_NA_Rewards_Active_Below50k_Single_NA (662562, 4)
Legacy_NA_Rewards_Active_Unknown_Unknown_NA (2303333, 4)
SOTF_NA_Rewards_Active_Unknown_Unknown_

In [25]:
list_to_upload=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/aggregatd_to_FB_directly_PlanB/*.csv")
len(list_to_upload)


18

In [31]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi




BigLotsAccount_id="act_271491453638620"

jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token="EAAkEB3l7ONYBALOwTjZA6ZA69KGshjjZCghKd2FhqZCVvcZCxcQZCglQshts2EZCZBccbvwUddxKlje4ZCz3w8qkLWICYdS8cZCAw2e8VZCs9VHEt7sg5gR6gc4o8TVV73OAABUf7zKqoXLyeREavx7Viz8ArwVQxsDfMVSJK7o5qOqiQZDZD"

def creatfbaudience(file_path,ACCESS_TOKEN,accountid,description,audience_name):
    '''
    input:
    file_path: str, full_path, basename as the audience name on FB
        make sure the columns contains the "email_address_hash", use this col only
    
    ACCESS_TOKEN: personal token #app id not needed
    accountid: account_id starting with "act_", followed by the id str
    as Nov. 20, 2019, latest verision is 'v5.0'
    '''
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v5.0')
    myAccount = AdAccount(accountid)
    fields = []
    fbname=audience_name
    params = {
      'name': fbname,
      'subtype': 'CUSTOM',
      'description': description,
      'customer_file_source': 'USER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    
    df=pd.read_csv(file_path,dtype=str,)
    listlen = df.shape[0]
    requestUrl = 'https://graph.facebook.com/v5.0/'+ fbname + '/users'
    
    chunck_size=50000
    i_counter=0
    for df_chunck in pd.read_csv(file_path,chunksize=chunck_size,dtype=str,usecols=["email_address_hash"]):
        payload = {}
        payload['schema'] = ['EMAIL']
        df_chunck=df_chunck.rename(columns={"email_address_hash":"EMAIL"})
        df_chunck=df_chunck.drop_duplicates()
        payload_data=df_chunck['EMAIL'].apply(lambda x: [x]).tolist()

        payload['data'] = payload_data
        params = {}
        params ={'payload':payload} 
        CustomAudience(fbname).create_user(
                      fields=fields,
                      params=params,
        )
        i_counter+=1
        print(audience_name,i_counter,datetime.datetime.now())
  

    return 'All done: '+str(datetime.datetime.now())

In [27]:
for local_file in list_to_upload:
    creatfbaudience(file_path=local_file,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id,
                    description=os.path.basename(local_file).split(".")[0],
                    audience_name=os.path.basename(local_file).split(".")[0]
                   )

1 2020-01-31 14:49:12.927752
2 2020-01-31 14:49:14.714219
1 2020-01-31 14:49:27.513752
2 2020-01-31 14:49:31.805085
3 2020-01-31 14:49:34.772560
4 2020-01-31 14:49:38.028048
5 2020-01-31 14:49:40.017299
6 2020-01-31 14:49:46.773457
7 2020-01-31 14:49:48.875603
8 2020-01-31 14:49:55.228729
9 2020-01-31 14:50:01.562315
10 2020-01-31 14:50:03.734260
11 2020-01-31 14:50:10.152927
12 2020-01-31 14:50:12.437729
13 2020-01-31 14:50:14.635697
14 2020-01-31 14:50:20.949330
15 2020-01-31 14:50:27.303897
16 2020-01-31 14:50:29.367223
17 2020-01-31 14:50:34.667585
18 2020-01-31 14:50:37.106663
19 2020-01-31 14:50:43.645626
20 2020-01-31 14:50:50.296389
21 2020-01-31 14:50:56.613926
22 2020-01-31 14:51:03.047490
23 2020-01-31 14:51:09.339932
24 2020-01-31 14:51:11.495516
25 2020-01-31 14:51:18.296340
26 2020-01-31 14:51:20.446608
1 2020-01-31 14:51:22.771290
1 2020-01-31 14:51:31.646970
2 2020-01-31 14:51:38.529508
3 2020-01-31 14:51:45.385636
4 2020-01-31 14:51:47.886780
5 2020-01-31 14:51:54.2431

9 2020-01-31 15:13:10.771170
10 2020-01-31 15:13:12.702107
11 2020-01-31 15:13:19.164784
12 2020-01-31 15:13:21.329560
13 2020-01-31 15:13:23.310269
14 2020-01-31 15:13:24.248126
1 2020-01-31 15:13:35.118495
2 2020-01-31 15:13:37.642792
3 2020-01-31 15:13:39.616255
4 2020-01-31 15:13:46.212505
5 2020-01-31 15:13:48.381547
6 2020-01-31 15:13:54.718358
7 2020-01-31 15:14:01.417937
8 2020-01-31 15:14:03.553419
9 2020-01-31 15:14:10.130289
10 2020-01-31 15:14:12.371452
11 2020-01-31 15:14:14.885041
12 2020-01-31 15:14:17.071607
13 2020-01-31 15:14:23.543514
14 2020-01-31 15:14:25.689713
15 2020-01-31 15:14:32.340715
16 2020-01-31 15:14:34.670473
17 2020-01-31 15:14:36.880411
18 2020-01-31 15:14:39.273806
19 2020-01-31 15:14:45.841912
20 2020-01-31 15:14:52.128226
21 2020-01-31 15:14:54.477438
22 2020-01-31 15:15:00.771081
23 2020-01-31 15:15:07.223391
24 2020-01-31 15:15:13.948410
25 2020-01-31 15:15:16.258057
26 2020-01-31 15:15:18.623230
27 2020-01-31 15:15:20.926175
28 2020-01-31 15:15:

In [29]:
# The control:
list_control_groups_all=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/*.csv")

list_control_groups_all=[x for x in list_control_groups_all if "C_all_quarterly_segments_RFM20Q1" in os.path.basename(x)]
list_control_groups_all

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/C_all_quarterly_segments_RFM20Q1.csv']

In [32]:
for local_file in list_control_groups_all:
    creatfbaudience(file_path=local_file,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id,
                    description=os.path.basename(local_file).split(".")[0],
                    audience_name=os.path.basename(local_file).split(".")[0]
                   )

C_all_quarterly_segments_RFM20Q1 1 2020-01-31 15:22:02.520921
C_all_quarterly_segments_RFM20Q1 2 2020-01-31 15:22:04.814105
C_all_quarterly_segments_RFM20Q1 3 2020-01-31 15:22:07.028237
C_all_quarterly_segments_RFM20Q1 4 2020-01-31 15:22:13.550477
C_all_quarterly_segments_RFM20Q1 5 2020-01-31 15:22:15.584740
C_all_quarterly_segments_RFM20Q1 6 2020-01-31 15:22:17.479239
C_all_quarterly_segments_RFM20Q1 7 2020-01-31 15:22:24.069261
C_all_quarterly_segments_RFM20Q1 8 2020-01-31 15:22:26.051652
C_all_quarterly_segments_RFM20Q1 9 2020-01-31 15:22:27.994230
C_all_quarterly_segments_RFM20Q1 10 2020-01-31 15:22:34.568865
C_all_quarterly_segments_RFM20Q1 11 2020-01-31 15:22:40.914222
C_all_quarterly_segments_RFM20Q1 12 2020-01-31 15:22:47.288627
C_all_quarterly_segments_RFM20Q1 13 2020-01-31 15:22:47.935971
